In [1]:
import pandas as pd
import catboost
import pickle
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from numba import jit
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
train = pd.read_csv("./csvs/train.csv")
test = pd.read_csv("./csvs/test.csv")

In [3]:
train.head(10)

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,evening_time,night_time,start_session,day_of_week,month,year,yearmonth,session_duration,sites_popularity_coeff,is_short
0,21669,safebrowsing.clients.google.com,2013-01-12 08:05:57,safebrowsing-cache.google.com,2013-01-12 08:05:57,empty,NaN,empty,NaN,empty,...,0,0,8,5,1,2013,201301,0.000000,2.948007,1
1,54843,safebrowsing.clients.google.com,2013-01-12 08:37:23,safebrowsing-cache.google.com,2013-01-12 08:37:23,safebrowsing.clients.google.com,2013-01-12 09:07:07,safebrowsing-cache.google.com,2013-01-12 09:07:09,empty,...,0,0,8,5,1,2013,201301,7.488294,2.948007,1
2,77292,www.apache.org,2013-01-12 08:50:13,www.apache.org,2013-01-12 08:50:14,download.eclipse.org,2013-01-12 08:50:15,www.apache.org,2013-01-12 08:50:15,www.apache.org,...,0,0,8,5,1,2013,201301,1.609438,0.016490,0
3,114021,www.webtide.com,2013-01-12 08:50:17,download.oracle.com,2013-01-12 08:50:17,www.caucho.com,2013-01-12 08:50:18,download.oracle.com,2013-01-12 08:50:18,www.webtide.com,...,0,0,8,5,1,2013,201301,1.386294,0.016490,0
4,146670,public.dhe.ibm.com,2013-01-12 08:50:20,jope.ow2.org,2013-01-12 08:50:20,download.oracle.com,2013-01-12 08:50:20,public.dhe.ibm.com,2013-01-12 08:50:21,jope.ow2.org,...,0,0,8,5,1,2013,201301,1.098612,0.008101,0
5,242171,master.dl.sourceforge.net,2013-01-12 08:50:22,public.dhe.ibm.com,2013-01-12 08:50:23,download.jboss.org,2013-01-12 08:50:23,www.apache.org,2013-01-12 08:50:23,public.dhe.ibm.com,...,0,0,8,5,1,2013,201301,1.386294,0.008101,0
6,57157,download.jboss.org,2013-01-12 08:50:25,public.dhe.ibm.com,2013-01-12 08:50:26,www.apache.org,2013-01-12 08:50:26,download.jboss.org,2013-01-12 08:50:26,dfn.dl.sourceforge.net,...,0,0,8,5,1,2013,201301,1.386294,0.001707,0
7,240201,www.apache.org,2013-01-12 08:50:28,public.dhe.ibm.com,2013-01-12 08:50:28,heanet.dl.sourceforge.net,2013-01-12 08:50:28,download.jboss.org,2013-01-12 08:50:29,www.apache.org,...,0,0,8,5,1,2013,201301,1.386294,0.001635,0
8,210686,www.apache.org,2013-01-12 08:50:31,kent.dl.sourceforge.net,2013-01-12 08:50:32,www.apache.org,2013-01-12 08:50:32,www.apache.org,2013-01-12 08:50:33,dfn.dl.sourceforge.net,...,0,0,8,5,1,2013,201301,1.791759,0.066223,0
9,98804,download.oracle.com,2013-01-12 08:50:37,www.apache.org,2013-01-12 08:50:37,download.oracle.com,2013-01-12 08:50:38,javadl-esd-secure.oracle.com,2013-01-12 08:50:49,gtssl-ocsp.geotrust.com,...,0,0,8,5,1,2013,201301,3.332205,0.867386,0


In [4]:
data = train.copy()

In [5]:
for i in range(1, 11):
    data[f"site{i}"] = data[f"site{i}"].astype("str")
    test[f"site{i}"] = test[f"site{i}"].astype("str")

In [6]:
data = data.drop([f"time{i}" for i in range(1, 11)], axis=1)
test = test.drop([f"time{i}" for i in range(1, 11)], axis=1)

In [8]:
data = data.drop("session_id", axis=1)
test = test.drop("session_id", axis=1)

In [10]:
data["session_duration"] = data["session_duration"].astype(np.int64)
data["sites_popularity_coeff"] = data["sites_popularity_coeff"].astype(np.int64)

test["session_duration"] = test["session_duration"].astype(np.int64)
test["sites_popularity_coeff"] = test["sites_popularity_coeff"].astype(np.int64)

In [11]:
data

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,evening_time,night_time,start_session,day_of_week,month,year,yearmonth,session_duration,sites_popularity_coeff,is_short
0,safebrowsing.clients.google.com,safebrowsing-cache.google.com,empty,empty,empty,empty,empty,empty,empty,empty,...,0,0,8,5,1,2013,201301,0,2,1
1,safebrowsing.clients.google.com,safebrowsing-cache.google.com,safebrowsing.clients.google.com,safebrowsing-cache.google.com,empty,empty,empty,empty,empty,empty,...,0,0,8,5,1,2013,201301,7,2,1
2,www.apache.org,www.apache.org,download.eclipse.org,www.apache.org,www.apache.org,www.webtide.com,download.oracle.com,javadl-esd-secure.oracle.com,www.caucho.com,www.apache.org,...,0,0,8,5,1,2013,201301,1,0,0
3,www.webtide.com,download.oracle.com,www.caucho.com,download.oracle.com,www.webtide.com,www.apache.org,public.dhe.ibm.com,www.webtide.com,www.apache.org,www.apache.org,...,0,0,8,5,1,2013,201301,1,0,0
4,public.dhe.ibm.com,jope.ow2.org,download.oracle.com,public.dhe.ibm.com,jope.ow2.org,master.dl.sourceforge.net,www.apache.org,download.eclipse.org,www.apache.org,public.dhe.ibm.com,...,0,0,8,5,1,2013,201301,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253528,ocsp.verisign.com,ocsp.verisign.com,ocsp.digicert.com,gtssl-ocsp.geotrust.com,ocsp.digicert.com,clients1.google.com,clients1.google.com,gtssl-ocsp.geotrust.com,www.mozilla.org,geo.mozilla.org,...,1,0,23,2,4,2014,201404,2,0,0
253529,dropbox.com,ocsp.godaddy.com,ocsp.godaddy.com,clients1.google.com,ocsp.verisign.com,dropbox.com,www.dropbox.com,dl-web.dropbox.com,www.dropbox.com,ajax.googleapis.com,...,1,0,23,2,4,2014,201404,5,0,0
253530,clients1.google.com,www.dropbox.com,javadl-esd-secure.oracle.com,javadl-esd-secure.oracle.com,www.dropbox.com,dl.javafx.com,c.woopic.com,orange.fr,www.orange.fr,c.orange.fr,...,1,0,23,2,4,2014,201404,3,0,0
253531,iapref.orange.fr,c.woopic.com,id.orange.fr,r.orange.fr,ocsp.godaddy.com,iapref.orange.fr,id.orange.fr,dl-web.dropbox.com,dl-web.dropbox.com,www.dropbox.com,...,1,0,23,2,4,2014,201404,4,0,0


In [12]:
data.dtypes

site1                     object
site2                     object
site3                     object
site4                     object
site5                     object
site6                     object
site7                     object
site8                     object
site9                     object
site10                    object
target                     int64
morning_time               int64
day_time                   int64
evening_time               int64
night_time                 int64
start_session              int64
day_of_week                int64
month                      int64
year                       int64
yearmonth                  int64
session_duration           int64
sites_popularity_coeff     int64
is_short                   int64
dtype: object

In [13]:
with open("./data/site.pkl", "rb") as fd:
    sites_mapping = pickle.load(fd)

In [14]:
for site in [f"site{i}" for i in range(1, 11)]:
    data[site] = data[site].map(sites_mapping)
    test[site] = test[site].map(sites_mapping)

In [19]:
data.head(5)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,evening_time,night_time,start_session,day_of_week,month,year,yearmonth,session_duration,sites_popularity_coeff,is_short
0,56,55,0,0,0,0,0,0,0,0,...,0,0,8,5,1,2013,201301,0,2,1
1,56,55,56,55,0,0,0,0,0,0,...,0,0,8,5,1,2013,201301,7,2,1
2,946,946,951,946,946,945,948,784,949,946,...,0,0,8,5,1,2013,201301,1,0,0
3,945,948,949,948,945,946,947,945,946,946,...,0,0,8,5,1,2013,201301,1,0,0
4,947,950,948,947,950,952,946,951,946,947,...,0,0,8,5,1,2013,201301,1,0,0


In [16]:
data[[f"site{i}" for i in range(1, 11)]] = data[[f"site{i}" for i in range(1, 11)]].fillna(0).astype('int')
test[[f"site{i}" for i in range(1, 11)]] = test[[f"site{i}" for i in range(1, 11)]].fillna(0).astype('int')

In [22]:
data.columns.to_list()

['site1',
 'site2',
 'site3',
 'site4',
 'site5',
 'site6',
 'site7',
 'site8',
 'site9',
 'site10',
 'target',
 'morning_time',
 'day_time',
 'evening_time',
 'night_time',
 'start_session',
 'day_of_week',
 'month',
 'year',
 'yearmonth',
 'session_duration',
 'sites_popularity_coeff',
 'is_short']

In [23]:
cat_features = [i for i in range(0, len(data.columns.to_list()) - 1)]

In [28]:
from sklearn.model_selection import train_test_split

X = data.drop("target", axis=1)
y = data["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)

In [29]:
y_val.sum()

464

In [30]:
X_train.dtypes

site1                     int64
site2                     int64
site3                     int64
site4                     int64
site5                     int64
site6                     int64
site7                     int64
site8                     int64
site9                     int64
site10                    int64
morning_time              int64
day_time                  int64
evening_time              int64
night_time                int64
start_session             int64
day_of_week               int64
month                     int64
year                      int64
yearmonth                 int64
session_duration          int64
sites_popularity_coeff    int64
is_short                  int64
dtype: object

In [42]:
clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.05, 
    eval_metric="Accuracy",
    loss_function="Logloss"
)


clf.fit(X_train, y_train, 
        cat_features=cat_features,
        eval_set=(X_val, y_val), 
        verbose=True,
        plot=True
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())
print('the val set accuracy is :{:.6f}'.format(accuracy_score(y_val, clf.predict(X_val))))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9909627	test: 0.9908494	best: 0.9908494 (0)	total: 97.2ms	remaining: 389ms
1:	learn: 0.9909627	test: 0.9908494	best: 0.9908494 (0)	total: 224ms	remaining: 337ms
2:	learn: 0.9909627	test: 0.9908494	best: 0.9908494 (0)	total: 385ms	remaining: 256ms
3:	learn: 0.9909627	test: 0.9908494	best: 0.9908494 (0)	total: 536ms	remaining: 134ms
4:	learn: 0.9909627	test: 0.9908494	best: 0.9908494 (0)	total: 629ms	remaining: 0us

bestTest = 0.9908493896
bestIteration = 0

Shrink model to first 1 iterations.
CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 5, 'learning_rate': 0.05, 'loss_function': 'Logloss', 'eval_metric': 'Accuracy'}
the val set accuracy is :0.990849


In [33]:
test.head(5)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,evening_time,night_time,start_session,day_of_week,month,year,yearmonth,session_duration,sites_popularity_coeff,is_short
0,21,0,0,0,0,0,0,0,0,0,...,0,0,17,3,5,2014,201405,0,2,1
1,23,66,63,2626,0,0,0,0,0,0,...,0,0,7,4,5,2014,201405,5,2,1
2,979,73,0,0,0,0,0,0,0,0,...,0,0,7,4,5,2014,201405,4,2,1
3,66,69,67,70,71,68,71,70,69,67,...,0,0,8,4,5,2014,201405,1,0,0
4,167,167,359,167,167,305,306,306,979,68,...,0,0,8,4,5,2014,201405,6,0,0


In [40]:
y_val

175841    0
93138     0
233578    0
246506    0
247795    0
         ..
140201    0
200758    0
193276    0
108580    0
89572     1
Name: target, Length: 50707, dtype: int64

In [35]:
sub = pd.DataFrame()
sub['target'] = clf.predict(X_val)
sub.to_csv('submit.csv', float_format='%.0f', index=False)